This notebook investigates using the polytope sampler for the 3d example from Paper1/2.

1. Polytope Sampler `../pilot_study_unfolding.ipynb`

The example is:
\begin{equation}
    \boldsymbol y = \boldsymbol x^* + \varepsilon, \; \; \varepsilon \sim N(\boldsymbol 0, \boldsymbol I_3), \; \; \boldsymbol x \geq 0,
\end{equation}
where $\boldsymbol x^* = \begin{pmatrix} 0 & 0 & 1 \end{pmatrix}^T$, and our functional of interest is defined as $\boldsymbol h = \begin{pmatrix} 1 & 1 & -1 \end{pmatrix}^T$.

Alternatively, we also examine points of the form $\boldsymbol x^* = \begin{pmatrix} t & t & 1 \end{pmatrix}^T$, where $t >0$.

__This notebook does the following__
1. Goes through one example of the polytope sampler applied to the 3d problem
2. Generates the chains for the interval computation
3. Analyzes the 3d output
4. Investigates quantiles on the space $\{x: h^Tx = h^T x^*, x \geq 0 \}$.
5. Test out a new importance sampler that move heavily samples near the parameter boundaries.

In [ ]:
from adaFuncCI.sample import ellipsoidSampler, polytopeSampler
from adaFuncCI.llr import llrSolver_3d
from adaFuncCI.inversion_intervals import solve_llr_fixed_y
from adaFuncCI.inversion_intervals import direct_inversion
from adaFuncCI.inversion_intervals import max_local_quantile_inversion
from adaFuncCI.optimize import osb_int, ssb_int
from adaFuncCI.max_quantile import maxQuantileRS
from adaFuncCI.utils import int_cover, percentile_ci_idx
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from time import time
from tqdm.notebook import tqdm

In [ ]:
import matplotlib
matplotlib.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 18})
plt.rcParams['text.usetex'] = True

# Define the LaTeX preamble to include multiple packages
plt.rcParams['text.latex.preamble'] = r'''
\usepackage{amsmath}
\usepackage{amssymb}
\usepackage{bm}
'''

# Generate Data

In [ ]:
# set fixed experiment settings
# x_star = np.array([0., 0., 1.])
t = 0.03
x_star = np.array([t, t, 1.])
h = np.array([1, 1, -1])
noise_distr = stats.norm
N = 1000  # number of data draws

# uncertainty parameters
alpha = 0.32
eta = 0.01
gamma = alpha - eta
print(gamma)

In [ ]:
# true functional
np.dot(h, x_star)

In [ ]:
# generate noise
np.random.seed(11211)
noise = noise_distr.rvs(size=(N, 3))
data = x_star + noise

In [ ]:
# estimate quantile at this point
llr_3d = llrSolver_3d()
np.percentile(llr_3d.solve_llr(x=x_star, E=stats.norm.rvs(size=30000).reshape((10000, 3))), q=68)

# 1 - One example of using polytope sampler versus ellipsoid sampler

"Bad" example: 746

"Normal" example: 0

Numerically annoying example: 452

In [ ]:
# define dictionary with sampler properties
mcmc_dict = {
    'N_hp': 6,
    'radius': 0.5,
    'polytope_type': 'eigen',
    'mcmc_alg': 'vaidya'
}

In [ ]:
def importance_sampler(
    data_i, M, x_center=np.zeros(3),
    eta=eta, K=np.identity(3), h=h,
    ap_gamma=0.5, ap_ord=0.25,
    mcmc_hp_dict=mcmc_dict
):
    """
    Wrapper around the Polytope algorithm that includes an extra
    accept/reject step.
    
    The accept probability is defined by exp(-ap_gamma * norm(x - x_center)_{ap_ord}).
    
    Parameters
    ----------
        data_i       (np arr) : data vector
        M            (int)    : number of total samples to draw
        x_center     (np arr) : center location of acceptance prob calc
        eta          (float)  : BB set prob
        K            (np arr) : forward model
        h            (np arr) : functional vector
        ap_gamma     (float)  : "accept-probability" gamma
        ap_ord       (float)  : "accept-probability" order of norm (should <1)
        mcmc_hp_dict (dict)   : hyperparameter for mcmc algo
        
    Returns
    -------
        final_sample (np arr) : complete sample
        
    """
    # declare sampler
    sampler = polytopeSampler(
        y=data_i,
        eta=eta,
        K=K,
        h=h,
        N_hp=mcmc_hp_dict['N_hp'],
        r=mcmc_hp_dict['radius'],
        random_seed=None,
        polytope_type=mcmc_hp_dict['polytope_type'],
        alg=mcmc_hp_dict['mcmc_alg'],
        disable_tqdm=True
    )
    
    num_samples = 0
    final_sample = np.zeros(shape=(M, K.shape[1]))
    prev_idx = 0
    curr_idx = 0
    while num_samples < M:

        # draw samples from polytope
        param_draws = sampler.sample_mixed_ensemble(M=M)

        # compute their accept reject probabilies
        accept_probs = np.exp(
            -ap_gamma * np.linalg.norm(param_draws - x_center, ord=ap_ord, axis=1)
        )

        # decide to accept/reject
        accept_reject = stats.bernoulli(accept_probs).rvs()
        
        # save accepted points
        curr_idx = prev_idx + accept_reject.sum()
        
        if curr_idx > M:
            curr_idx = M
        
        final_sample[prev_idx:curr_idx, :] = param_draws[
            accept_reject==1, :
        ][:(curr_idx - prev_idx)]
        
        # updates
        prev_idx = curr_idx
        num_samples += accept_reject.sum()

    return final_sample

In [ ]:
# generate sample from the importance sampler
OBS_IDX = 0
sample_is = importance_sampler(
    data_i=data[OBS_IDX], M=16000, ap_gamma=0.75, ap_ord=0.5
)

In [ ]:
# draws from the original sampler
sampler = polytopeSampler(
    y=data[OBS_IDX],
    eta=eta,
    K=np.identity(3),
    h=h,
    N_hp=6,
    r=0.5,
    random_seed=None,
    polytope_type='eigen',
    alg='vaidya',
    disable_tqdm=False
)

# draw parameter sample in the BB set
param_draws = sampler.sample_mixed_ensemble(M=16000)

In [ ]:
# create curve in x1/x2 plane
x_grid = np.linspace(0, 4, num=200)
x2_vals = data[OBS_IDX][1] + np.sqrt(stats.chi2(3).ppf(1 - eta) - (data[OBS_IDX][0] - x_grid) ** 2 - data[OBS_IDX][2] ** 2)
x2_nz = x2_vals >= 0

# create the curve in x1/x3 plane
x3_vals = data[OBS_IDX][2] + np.sqrt(stats.chi2(3).ppf(1 - eta) - (data[OBS_IDX][0] - x_grid) ** 2 - data[OBS_IDX][1] ** 2)
x3_nz = x3_vals >= 0

# create curve in the x2/x3 plane
x23_vals = data[OBS_IDX][2] + np.sqrt(stats.chi2(3).ppf(1 - eta) - data[OBS_IDX][0] ** 2 - (data[OBS_IDX][1] - x_grid) ** 2)
x23_nz = x23_vals >= 0

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(12, 4), sharex=True, sharey=True)

# 1 + 2
ax[0].scatter(sample_is[:, 0], sample_is[:, 1], alpha=0.15, s=5)
ax[0].scatter([x_star[0]], [x_star[1]], color='red')
ax[0].plot(x_grid[x2_nz], x2_vals[x2_nz], color='orange', label='Berger--Boos Set')
ax[0].axhline(2.38890183e+00, color='orange', xmin=0, xmax=(3.88756344e+00)/4, linestyle=':', label='Polytope Boundary')
ax[0].axvline(3.88756344e+00, color='orange', ymin=0, ymax=(2.38890183e+00)/4, linestyle=':')

# 1 + 3
ax[1].scatter(sample_is[:, 0], sample_is[:, 2], alpha=0.15, s=5)
ax[1].scatter([x_star[0]], [x_star[2]], color='red')
ax[1].plot(x_grid, x3_vals, color='orange')
ax[1].axhline(4.05231276e+00, color='orange', xmin=0, xmax=0.96, linestyle=':', label='Polytope Boundary')
ax[1].axvline(3.88756344e+00, color='orange', ymin=0, ymax=0.96, linestyle=':')

# 2 + 3
ax[2].scatter(sample_is[:, 1], sample_is[:, 2], alpha=0.15, s=5)
ax[2].scatter([x_star[1]], [x_star[2]], color='red')
ax[2].plot(x_grid, x23_vals, color='orange')
ax[2].axhline(4.05231276e+00, color='orange', xmin=0, xmax=2.38890183e+00/4, linestyle=':', label='Polytope Boundary')
ax[2].axvline(2.38890183e+00, color='orange', ymin=0, ymax=0.96, linestyle=':')

# other plot features
ax[0].set_xlabel(r'$x_1$')
ax[0].set_ylabel(r'$x_2$')
ax[1].set_xlabel(r'$x_1$')
ax[1].set_ylabel(r'$x_3$')
ax[2].set_xlabel(r'$x_2$')
ax[2].set_ylabel(r'$x_3$')
# ax[0].set_title(r'$x_1$ by $x_2$')
# ax[1].set_title(r'$x_1$ by $x_3$')
# ax[2].set_title(r'$x_2$ by $x_3$')

ax[0].legend()

plt.tight_layout()

plt.show()

In [ ]:
# look at distribution of functional values
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(10, 3), sharex=True, sharey=True)

# importance sampler
ax[0].hist(sample_is @ h, bins=40, histtype='step')
ax[0].axvline(np.dot(h, x_star), linestyle='--', color='gray')
ax[0].set_title('Importance Sampler')

# original sampler
ax[1].hist(param_draws @ h, bins=40, histtype='step')
ax[1].axvline(np.dot(h, x_star), linestyle='--', color='gray')
ax[1].set_title('Original Sampler')

plt.show()

#### Look at the distribution of quantiles

In [ ]:
# create llr objects
llr = llrSolver_3d()
max_q_rs = maxQuantileRS(
    X_train=sample_is,
    llr_solver=llr,
    distr=stats.norm,
    q=gamma,
    disable_tqdm=False
)
max_q_rs_orig = maxQuantileRS(
    X_train=param_draws,
    llr_solver=llr,
    distr=stats.norm,
    q=gamma,
    disable_tqdm=False
)

In [ ]:
# compute quantiles
maxq = max_q_rs.estimate(
    num_samp=10000,
    random_seeds=None
)

maxq_orig = max_q_rs_orig.estimate(
    num_samp=10000,
    random_seeds=None
)

In [ ]:
# create sorted data
sort_idx = np.argsort(sample_is @ h)
qoi_vals_samp = (sample_is @ h)[sort_idx]
q_hat_samp = max_q_rs.max_quantiles[sort_idx]

# create sorted data for the original sampler
sort_idx_orig = np.argsort(param_draws @ h)
qoi_vals_samp_orig = (param_draws @ h)[sort_idx_orig]
q_hat_samp_orig = max_q_rs_orig.max_quantiles[sort_idx_orig]

In [ ]:
# solve for the LLR at each sampled functional value
llr_vals_qoi_test = solve_llr_fixed_y(
    qoi_vals=sample_is @ h,
    y=data[OBS_IDX],
    K=np.identity(3),
    h=h,
    disable_tqdm=False
)

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(12, 6), sharex=True, sharey=True)

# new sampler
ax[0].scatter(qoi_vals_samp, q_hat_samp, alpha=0.15, s=5)
ax[0].axvline(np.dot(h, x_star), linestyle='--', color='gray')
ax[0].plot(qoi_vals_samp, llr_vals_qoi_test[sort_idx], alpha=0.6, color='black')
# ax[0].plot(qoi_vals_samp[T - 1:], max_q_pred_samp, color='red')
ax[0].set_ylim(0.6, 1.3)
ax[0].set_title('Importance-Like Sampler')

# old sampler
ax[1].scatter(qoi_vals_samp_orig, q_hat_samp_orig, alpha=0.15, s=5)
ax[1].axvline(np.dot(h, x_star), linestyle='--', color='gray', label='True Functional Value')
ax[1].plot(qoi_vals_samp, llr_vals_qoi_test[sort_idx], alpha=0.6, color='black', label=r'$\lambda(\mu, \boldsymbol y)$')
ax[1].set_title('Polytope Sampler')

# other plot attributes
ax[1].legend()
ax[0].set_xlabel(r'Functional Value ($\mu$)')
ax[1].set_xlabel(r'Functional Value ($\mu$)')
ax[0].set_ylabel('Computed Quantile')
plt.tight_layout()
plt.show()

In [ ]:
# --- MQ direct
mq_direct = direct_inversion(
    qoi_vals=sample_is @ h,
    llr_vals_qoi=llr_vals_qoi_test,
    q_hat_vals=max_q_rs.max_quantiles,
    local=False
)

# --- MQ Opt
mq_opt = osb_int(
    y=data[OBS_IDX], q=max_q_rs.max_quantiles.max(), K=np.identity(3), h=h
)

# --- MQmu param
mq_mu_param = direct_inversion(
    qoi_vals=sample_is @ h,
    llr_vals_qoi=llr_vals_qoi_test,
    q_hat_vals=max_q_rs.max_quantiles,
    local=True
)

# --- MQmu func
mq_mu_func = max_local_quantile_inversion(
    qoi_vals=sample_is @ h,
    llr_vals_qoi=llr_vals_qoi_test,
    q_hat_vals=max_q_rs.max_quantiles,
    method='rolling',
    hyperparams={'T': 10, 'center': True}
)[0]

# --- OSB Interval
osb = osb_int(
    y=data[OBS_IDX], q=stats.chi2(1).ppf(1 - alpha), K=np.identity(3), h=h
)

In [ ]:
print('--- Intervals ---')
print(f'MQ Direct : ({mq_direct[0]:.2f}, {mq_direct[1]:.2f})')
print(f'MQ Opt    : ({mq_opt[0]:.2f}, {mq_opt[1]:.2f})')
print(f'MQmu Param: ({mq_mu_param[0]:.2f}, {mq_mu_param[1]:.2f})')
print(f'MQmu Func : ({mq_mu_func[0]:.2f}, {mq_mu_func[1]:.2f})')
print(f'OSB       : ({osb[0]:.2f}, {osb[1]:.2f})')

In [ ]:
# create sorted data
sort_idx = np.argsort(sample_is @ h)
qoi_vals_samp = (sample_is @ h)[sort_idx]
q_hat_samp = max_q_rs.max_quantiles[sort_idx]
llr_vals_samp = llr_vals_qoi_test[sort_idx]

# create rolling max
T = 10
max_q_pred_samp = pd.Series(q_hat_samp).rolling(T, center=True).max().dropna()
llr_vals_roll = llr_vals_samp[T - 1:].copy()

In [ ]:
plt.figure(figsize=(5, 3))
sort_idx = np.argsort(sample_is @ h)
qoi_vals_samp = (sample_is @ h)[sort_idx]
llr_vals_samp = llr_vals_qoi_test[sort_idx]
plt.scatter(sample_is @ h, max_q_rs.max_quantiles, alpha=0.05)
plt.plot(qoi_vals_samp, llr_vals_samp)
plt.plot(qoi_vals_samp[T - 1:], max_q_pred_samp, color='red')
plt.axvline(-1, linestyle='--', color='gray')
plt.ylim(0.6, 1.3)
# plt.xlim(-1.1, -.95)
plt.show()

# 2 - Generate Chains for the Parallel Experiment

### Samples via Importance Sampling

__NOTE__: this is too slow to do locally. Look at `../parallel_scripts/3d_experiments/importance_sampler_X.py`.

In [ ]:
# M = 16000
# AP_GAMMA = 0.75
# AP_ORD = 0.5
# param_draws_all = np.zeros(shape=(1000, M, 3))

# for i in tqdm(range(1000)):

#     # generate sample
#     param_draws_all[i, :, :] = importance_sampler(
#         data_i=data[i],
#         M=M,
#         ap_gamma=AP_GAMMA, ap_ord=AP_ORD,
#         eta=eta, K=np.identity(3), h=h,
#         mcmc_hp_dict=mcmc_dict
#     )

# 3 - Analyze Parallel Output

__NOTE__: the files read in below are too large to store in the github repository. Please reach out to `mcstanle@alumni.cmu.edu` to gain file access.

In [ ]:
from statsmodels.stats.proportion import proportion_confint

In [ ]:
# make key for interval types
interval_type_key = {
    0: "Global Inverted",
    1: "Global Optimized",
    2: "Sliced Inverted",
    3: "Sliced Optimized",
    4: "OSB"
}

In [ ]:
# read in the computed quantiles
with open(
#     '../data/3d_experiments/numObs_1000_num_param_16000_num_quant_16000_alpha0.32_eta0.01_rolllingT10mixed_polytope_sampler.npz',
#     '../data/3d_experiments/numObs_1000_num_param_16000_num_quant_16000_alpha0.32_eta0.01_rolllingT10mixed_polytope_sampler_t0.1.npz',
#     '../data/3d_experiments/numObs_1000_num_param_16000_num_quant_10000_alpha0.32_eta0.01_rolllingT10_importance_sampler_t0.03.npz',
    '../data/3d_experiments/numObs_1000_num_param_16000_num_quant_10000_alpha0.32_eta0.01_rolllingT10_importance_sampler_t0.03_bbcalib_center_roll.npz',
    'rb'
) as f:
    exp_obj = np.load(f)
    exp_intervals = exp_obj['intervals']
    exp_qoi_vals = exp_obj['qoi_vals']
    exp_llr_vals = exp_obj['llr_vals']
    exp_q_hat_vals = exp_obj['q_hat_vals']

In [ ]:
# number of observations
NUM_OBS = exp_intervals.shape[0]
NUM_INTS = exp_intervals.shape[1]

#### Length

In [ ]:
interval_lengths = exp_intervals[:, :, 1] - exp_intervals[:, :, 0]

for i in range(exp_intervals.shape[1]):
    print(f'{interval_type_key[i]}: Estimated Length: {interval_lengths.mean(axis=0)[i]}')

In [ ]:
plt.figure(figsize=(8, 5))
plt.grid(True, axis='y')

plt.bar(x=np.arange(5), height=interval_lengths.mean(axis=0))
plt.vlines(
    x=np.arange(5),
    ymin=interval_lengths.mean(axis=0) - stats.norm.ppf(0.975) * (interval_lengths.std(axis=0) / np.sqrt(NUM_OBS)),
    ymax=interval_lengths.mean(axis=0) + stats.norm.ppf(0.975) * (interval_lengths.std(axis=0) / np.sqrt(NUM_OBS)),
    color='orange'
)

plt.xticks(ticks=np.arange(5), labels=[interval_type_key[i] for i in range(5)], rotation=15)
plt.ylim(2.6, 3.05)
plt.title('Estimated Expected Length')
plt.tight_layout()
plt.show()

#### Coverage

In [ ]:
# compute coverage
coverage = np.zeros(shape=(NUM_OBS, NUM_INTS))
for i in range(NUM_OBS):
    for j in range(NUM_INTS):
#         coverage[i, j] = int_cover(mu_true=np.dot(h, np.array([0, 0, 1])), interval=exp_intervals[i, j, :])
        coverage[i, j] = int_cover(mu_true=np.dot(h, x_star), interval=exp_intervals[i, j, :])
        
for i in range(NUM_INTS):
    print(f'{interval_type_key[i]}: Estimated Coverage: {coverage.mean(axis=0)[i]}')

In [ ]:
coverage_cis = np.zeros(shape=(NUM_INTS, 2))
for i in range(NUM_INTS):
    coverage_cis[i, :] = proportion_confint(
        coverage.mean(axis=0)[i] * NUM_OBS, NUM_OBS, alpha=0.05, method='beta'
    )

print(coverage_cis)

In [ ]:
plt.figure(figsize=(8, 5))
plt.grid(True, axis='y')
plt.bar(x=np.arange(5), height=coverage.mean(axis=0))

plt.vlines(x=np.arange(5), ymin=coverage_cis[:, 0], ymax=coverage_cis[:, 1], color='orange')

plt.xticks(ticks=np.arange(5), labels=[interval_type_key[i] for i in range(5)], rotation=15)
plt.axhline(0.68, linestyle='--', color='gray')
plt.ylim(0.6, 0.74)
plt.title('Estimated Coverage')
plt.tight_layout()
plt.show()